# i. Data Imports

In [1]:
import re
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nfl_data_py as nfl

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
YEARS = range(2010, 2025)

In [3]:
data_root = Path('../data/raw/pbp')
data_root.mkdir(parents=True, exist_ok=True)

def load_or_cache_pbp_data(year):
    cache_file = data_root / f"{year}.parquet"
    
    if cache_file.exists():
        print(f"Loading {year} data from cache...")
        return pd.read_parquet(cache_file)
    else:
        print(f"Downloading {year} data from NFL API...")
        df = nfl.import_pbp_data([year])
        
        # Cache the data
        df.to_parquet(cache_file, index=False)
        print(f"Cached {year} data to {cache_file}")
        
        return df

def load_all_pbp_data(years):
    dataframes = []
    
    for year in years:
        df = load_or_cache_pbp_data(year)
        dataframes.append(df)
    
    print(f"Combining data for {len(dataframes)} seasons...")
    combined_df = pd.concat(dataframes, ignore_index=True)
    print(f"Total rows: {len(combined_df):,}")
    
    return combined_df

In [4]:
pbp_data = load_all_pbp_data(YEARS)
pbp_data.head()

Loading 2010 data from cache...
Loading 2011 data from cache...
Loading 2012 data from cache...
Loading 2013 data from cache...
Loading 2014 data from cache...
Loading 2012 data from cache...
Loading 2013 data from cache...
Loading 2014 data from cache...
Loading 2015 data from cache...
Loading 2016 data from cache...
Loading 2015 data from cache...
Loading 2016 data from cache...
Loading 2017 data from cache...
Loading 2018 data from cache...
Loading 2017 data from cache...
Loading 2018 data from cache...
Loading 2019 data from cache...
Loading 2020 data from cache...
Loading 2019 data from cache...
Loading 2020 data from cache...
Loading 2021 data from cache...
Loading 2022 data from cache...
Loading 2021 data from cache...
Loading 2022 data from cache...
Loading 2023 data from cache...
Loading 2024 data from cache...
Loading 2023 data from cache...
Loading 2024 data from cache...
Combining data for 15 seasons...
Combining data for 15 seasons...
Total rows: 721,564
Total rows: 721,56

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,td_player_name,td_player_id,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,vegas_wpa,vegas_home_wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passer_player_id,passer_player_name,passing_yards,receiver_player_id,receiver_player_name,receiving_yards,rusher_player_id,rusher_player_name,rushing_yards,lateral_receiver_player_id,lateral_receiver_player_name,lateral_receiving_yards,lateral_rusher_player_id,lateral_rusher_player_name,lateral_rushing_yards,lateral_sack_player_id,lateral_sack_player_name,interception_player_id,interception_player_name,lateral_interception_player_id,lateral_interception_player_name,punt_returner_player_id,punt_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tack

In [5]:
final_cols = [
    'season',
    'game_id', 
    'play_id', 
    'game_date', 
    'start_time',
    'time_of_day',
    'stadium',
    'game_stadium',
    'stadium_id',
    'weather',
    'location',
    'roof',
    'surface',
    'temp',
    'wind',
    'season_type', 
    'posteam', 
    'defteam',
    'kicker_player_name',
    'kicker_player_id',
    'yardline_100',
    'qtr',
    'quarter_seconds_remaining',
    'game_seconds_remaining',
    'posteam_score',
    'defteam_score',
    'score_differential',
    'iced_kicker',
    'field_goal_result',
]

fg_data = (
    pbp_data
    .sort_values(by=['game_id', 'play_id'])
    .assign(
        prev_row_game_id=lambda x: x.game_id.shift(1),
        prev_row_timeout_called=lambda x: x.timeout.shift(1),
        prev_row_timeout_team=lambda x: x.timeout_team.shift(1),
    )
    .assign(
        iced_kicker=lambda x: (
            (x.prev_row_game_id == x.game_id) &
            (x.prev_row_timeout_called == True) &
            (x.prev_row_timeout_team != x.posteam) &
            (x.prev_row_timeout_team.notnull() & x.prev_row_timeout_team != '')
        )
    )
    .query('field_goal_attempt==1')
    [final_cols]
)

fg_data.head()

,season,game_id,play_id,game_date,start_time,time_of_day,stadium,game_stadium,stadium_id,weather,location,roof,surface,temp,wind,season_type,posteam,defteam,kicker_player_name,kicker_player_id,yardline_100,qtr,quarter_seconds_remaining,game_seconds_remaining,posteam_score,defteam_score,score_differential,iced_kicker,field_goal_result
14,2010,2010_01_ARI_STL,350.0,2010-09-12,"9/12/10, 16:15:48",2010-09-12T20:24:49Z,Edward Jones Dome,Edward Jones Dome,STL00,"Temp: ° F, Wind: mph",Home,dome,astroplay,NaN,NaN,REG,LA,ARI,Josh.Brown,00-0021940,16.0,1.0,581.0,3281.0,0.0,0.0,0.0,False,blocked
42,2010,2010_01_ARI_STL,1018.0,2010-09-12,"9/12/10, 16:15:48",2010-09-12T20:55:52Z,Edward Jones Dome,Edward Jones Dome,STL00,"Temp: ° F, Wind: mph",Home,dome,astroplay,NaN,NaN,REG,ARI,LA,J.Feely,00-0019770,4.0,2.0,837.0,2637.0,0.0,0.0,0.0,False,made
66,2010,2010_01_ARI_STL,1602.0,2010-09-12,"9/12/10, 16:15:48",2010-09-12T21:22:19Z,Edward Jones Dome,Edward Jones Dome,STL00,"Temp: ° F, Wind: mph",Home,dome,astroplay,NaN,NaN,REG,LA,ARI,Josh.Brown,00-0021940,28.0,2.0,333.0,2133.0,0.0,3.0,-3.0,False,made
117,2010,2010_01_ARI_STL,2826.0,2010-09-12,"9/12/10, 16:15:48",2010-09-12T22:26:51Z,Edward Jones Dome,Edward Jones Dome,STL00,"Temp: ° F, Wind: mph",Home,dome,astroplay,NaN,NaN,REG,LA,ARI,Josh.Brown,00-0021940,7.0,3.0,512.0,1412.0,10.0,10.0,0.0,False,made
209,2010,2010_01_ATL_PIT,280.0,2010-09-12,"9/12/10, 13:03:09",2010-09-12T17:10:58Z,Acrisure Stadium,Heinz Field,PIT00,"Cloudy Temp: 65° F, Humidity: 67%, Wind: From ...",Home,outdoors,grass,65.0,9.0,REG,PIT,ATL,J.Reed,00-0020737,34.0,1.0,477.0,3177.0,0.0,0.0,0.0,False,made


In [6]:
fg_data.roof.value_counts()

outdoors    11100
dome         2344
closed       2059
open          284
Name: roof, dtype: int64

In [7]:
def infer_rain_chance(text):
    if text is None or pd.isna(text):
        return 0

    text = text.lower()

    # Try to extract an explicit % chance
    percent_match = re.search(r'(\d+)% chance of rain', text)
    if percent_match:
        return int(percent_match.group(1))

    # Heuristics
    if 'light rain' in text:
        return 75
    elif 'rainy' in text or 'rain' in text:
        return 90
    elif 'occasional rain' in text:
        return 70
    elif 'cloudy / rainy' in text:
        return 60
    elif 'cloudy, chance of rain' in text:
        return 30
    else:
        return np.nan
    
def parse_snow_severity(text):
    if text is None or pd.isna(text):
        return 0
    
    text = text.lower()

    # 4 - Severe snow conditions
    if 'heavy snow' in text or 'heavy lake effect snow' in text:
        return 4
    if re.search(r'\b(blizzard|whiteout|3 to 5 inches|4 to 6 inches|accumulating \d)', text):
        return 4

    # 3 - Normal snow
    if re.search(r'\bsnow\b', text) and 'light' not in text and 'showers' not in text and 'flurries' not in text:
        return 3

    # 2 - Light snow, snow showers
    if 'light snow' in text or 'snow showers' in text or 'snow shower' in text:
        return 2

    # 1 - Minimal snow
    if 'flurries' in text or 'chance of snow' in text or 'few light flakes' in text:
        return 1

    # 0 - No snow
    return 0

def parse_wind(weather_str):
    """Extracts base wind speed from weather string."""
    if not isinstance(weather_str, str):
        return None
    
    weather_str = weather_str.lower().replace("m.p.h.", "mph").replace("m.p.h", "mph").replace(".", "")
    
    # Pattern 1: "Wind: N 17 mph" or "Wind: South 14 mph"
    match = re.search(r'wind:\s*[a-z ]*\s*(\d+)\s*mph', weather_str)
    if match:
        return int(match.group(1))
    
    # Pattern 2: "wind: w 11 g 21 mph" (extract first number)
    match = re.search(r'wind:\s*[a-z\s]*?(\d+)\s*g\s*\d+\s*mph', weather_str)
    if match:
        return int(match.group(1))
    
    # Pattern 3: "wind: nw 19 with gusts to 26 mph" (extract first number)
    match = re.search(r'wind:\s*[a-z\s]*?(\d+)[^\d]+gusts?\s*(?:to)?\s*\d+\s*mph', weather_str)
    if match:
        return int(match.group(1))
    
    # Pattern 4: "wind: south 40+ mph" or "wind: from sse 71 mph"
    match = re.search(r'wind:\s*(?:from\s*)?[a-z\s]+(\d+)\+?\s*mph', weather_str)
    if match:
        return int(match.group(1))
    
    # Pattern 5: "wind: 20 mph" (standalone number)
    match = re.search(r'wind:\s*(\d+)\s*mph', weather_str)
    if match:
        return int(match.group(1))
    
    # Pattern 6: "wind: north west 20 mph" (direction + number)
    match = re.search(r'wind:\s*[a-z\s]+\s+(\d+)\s*mph', weather_str)
    if match:
        return int(match.group(1))
    
    return None

def parse_wind_gust(weather_str):
    """Enhanced function to extract gust speed from various weather string patterns."""
    if not isinstance(weather_str, str):
        return None
    
    # Normalize the string
    weather_str = weather_str.lower()
    weather_str = (weather_str.replace("m.p.h.", "mph")
                  .replace("m.p.h", "mph")
                  .replace(".", "")
                  .replace("up to", "to"))  # Standardize "gusts up to" to "gusts to"
    
    # Try multiple gust patterns in order of specificity
    patterns = [
        # Pattern 1: "gusts to 30-35 mph" (range) - groups: (30, 35)
        (r'gusts? (?:to )?(\d+)\s*-\s*(\d+)\s*mph', 2),
        # Pattern 2: "gusting to 26 mph" or "gusting to 17mph" - group: (26)
        (r'gusting to (\d+)\s*mph', 1),
        # Pattern 3: "wind gusts up to 29 mph" - group: (29)
        (r'(?:wind\s+)?gusts? to (\d+)\s*mph', 1),
        # Pattern 4: "West 15 gusts up to 25 mph" - group: (25)
        (r'wind: [a-z]+\s+\d+\s+gusts? (?:up )?to (\d+)\s*mph', 1),
        # Pattern 5: "winds in the 15-20 with gusts up to 30-35" - group: (30)
        (r'gusts? (?:up )?to (\d+)', 1),
        # Pattern 6: "gusty winds" (generic) - no groups
        (r'gusty winds?', 0)
    ]
    
    gust = None
    
    for pattern, groups in patterns:
        match = re.search(pattern, weather_str)
        if match:
            if groups == 2:  # Handle range case
                gust = int(match.group(1))  # Take the lower bound
                break
            elif groups == 1:  # Single number case
                gust = int(match.group(1))
                break
            elif groups == 0:  # "gusty winds" case
                gust = 25  # Default value for gusty winds
                break
    
    # Handle "gale force" wind phrases (39-54 mph NWS definition)
    if 'gale' in weather_str:
        gust = max(gust or 0, 47)  # Use midpoint of gale range
    
    # If no gust found but "gusty" mentioned, use default
    if gust is None and ('gusty' in weather_str or 'gusting' in weather_str):
        gust = 25
    
    return gust

fg_data = (
    fg_data
    .assign(
        chance_of_rain=np.where(
            fg_data.roof.isin(['open', 'outdoors']),
            fg_data.weather.apply(infer_rain_chance),
            0
        ),
        snow_severity=np.where(
            fg_data.roof.isin(['open', 'outdoors']),
            fg_data.weather.apply(parse_snow_severity),
            0
        ),
        wind_speed=np.where(
            fg_data.roof.isin(['open', 'outdoors']),
            fg_data.weather.apply(parse_wind),
            0
        ),
        wind_gust=np.where(
            fg_data.roof.isin(['open', 'outdoors']),
            fg_data.weather.apply(parse_wind_gust),
            0
        )
    )
)